# Add feature

#### This notebook demonstrates that how to segment any given dataset based on the types of workload included in it.
#### This notebook also includes examples of different time series types, and how to inteprate the *characteristic scores*.

In [1]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import heapq

In [2]:
# important window definition
prediction_length = 48
context_length = 72
day = 24
week = 148
month = 720
year = 8760

freq = 'H'

### Prepare time series

In [3]:
# get the Time Series data and corresponding wiki project name
def get_ts_and_name(data_location,label_location,freq):
    df_ts = pd.read_json(data_location, lines=True)
    num_pt = min(len(df_ts.iloc[1, 1]), 100000)
    print('use first ', num_pt, ' points in a time series')
    num_ts = len(df_ts)

    time_series_wiki = []
    for k in range(num_ts):
        t0 = df_ts.iloc[k, 0]
        data = df_ts.iloc[k, 1][:num_pt]
        index = pd.DatetimeIndex(start=t0, freq=freq, periods=num_pt)
        time_series_wiki.append(pd.Series(data=data, index=index))
        
    with open(label_location) as f:
        wp_list = f.read().splitlines()  
        
    return time_series_wiki, wp_list

# get the Time Series data and corresponding wiki project name
def get_ts(data_location,freq):
    df_ts = pd.read_json(data_location, lines=True)
    num_pt = min(len(df_ts.iloc[1, 1]), 100000)
    print('use first ', num_pt, ' points in a time series')
    num_ts = len(df_ts)

    time_series_wiki = []
    for k in range(num_ts):
        t0 = df_ts.iloc[k, 0]
        data = df_ts.iloc[k, 1][:num_pt]
        index = pd.DatetimeIndex(start=t0, freq=freq, periods=num_pt)
        time_series_wiki.append(pd.Series(data=data, index=index))       
    return time_series_wiki

In [4]:
# label_location = 'wp_full-20180101-20190101_get.txt'    
# data_location = 'test_1year.json'
# 

# time_series_wiki = get_ts(data_location,freq)

### Specify I/O

In [5]:
input_file = 'train_nan.json'
output_file = "train_nan_cat1.json"

time_series_wiki_1 = get_ts(input_file,freq)
raw_jsons = []
with open(input_file) as f:
    for line in f:
        raw_jsons.append(json.loads(line))
# print(raw_jsons[1])

use first  17544  points in a time series


In [6]:
time_series_wiki_1[322].isnull().sum()

8311

In [7]:
tm_0 = time_series_wiki_1[322].replace(to_replace=np.nan, value=0, inplace = False)
tm_0.isnull().sum()

0

In [8]:
#time_series_wiki_1[322]

In [9]:
#time_series_wiki_1[322].isnull()

In [10]:
# with s3filesystem.open(os.path.join(S3_DATA_PATH, "test", "train_data.json"), 'wb') as fp:
#     for ts in timeseries_training:
#         fp.write(series_to_jsonline(ts).encode(encoding))
#         fp.write('\n'.encode(encoding))

### The time series characterization tool based on discrete Fourier transform.

In [11]:
# discrete Fourier transform with FFT
def discrete_ft(x,window):
    # important variables are written explicitly
    y = x.values # signal

    Fs = 1 # sampling rate, in our case let's use 1 Hour^-1
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range

    # we only keep the positive frequency up to the Nyquist = 1/2*Fs
    cycle = frq[range(1,int(n/2))] * window # one side frequency range

    Y = np.fft.fft(y)/n # fft computing and normalization
    Y = Y[range(1, int(n/2))] # again, spectrum corrresponding to the positive half

    fig, ax = plt.subplots(2, 1, figsize = (10,5))
    ax[0].plot(x)
    ax[0].set_xlabel('Time')
    ax[0].set_ylabel('network traffic')
    ax[1].plot(cycle,abs(Y),'r') # plotting the spectrum
    ax[1].set_xlabel('cycles per '+str(window)+' hours')
    ax[1].set_ylabel('network frequency spectrum')

In [12]:
# time series characterization with FFT
import heapq
def characterize_ts(ts, window):
    y = ts.values  # signal
    Fs = 1  # sampling rate, in our case let's use 1 Hour^-1
    n = len(y)  # length of the signal
    k = np.arange(n)
    T = n / Fs
    frq = k / T  # two sides frequency range
    # We only keep the positive frequency up to the Nyquist = 1/(2*dT), dT = sampling interval.
    cycle = frq[range(1, int(n / 2))] * window  # one side frequency range

    Y = np.fft.fft(y) / n  # fft computing and normalization
    Y = Y[range(1, int(n / 2))]  # again, spectrum corrresponding to the positive half
    yabs = np.abs(Y)

    # Locate the largest 15 peaks, use them to characterise the time series.
    indx = heapq.nlargest(15, range(len(yabs)), yabs.__getitem__)
    amp = heapq.nlargest(15, yabs)

    mean = yabs.mean()
    std = yabs.std()
    cyc_hday = 2.0
    cyc_day = cyc_hday / 2.0
    cyc_week = cyc_day / 7.0
    cyc_month = cyc_day / 30.0

    comp = lambda a, b: np.abs(a / b - 1) < 0.05
    ts_type = ['trend', 'hDay', 'Day', 'Week', 'Month', 'DayImpulse', 'spike']
    report_list = [0] * 8
    for counter, value in enumerate(indx):
        # Here we define a peak in frequency domain.
        if amp[counter] > (mean + 3 * std):
            amp_norm = (amp[counter] - mean) / std
            if cycle[value] < 0.01:
                # Trend (increasing, decreasing, gaussian pulse).
                report_list[0] = max(amp_norm, report_list[0])
            elif comp(cycle[value], cyc_hday):
                report_list[1] = max(amp_norm, report_list[1])
            elif comp(cycle[value], cyc_day):
                report_list[2] = max(amp_norm, report_list[2])
            elif comp(cycle[value], cyc_week):
                report_list[3] = max(amp_norm, report_list[3])
            elif comp(cycle[value], cyc_month):
                report_list[4] = max(amp_norm, report_list[4])

    if sum(report_list[:5]) > 0:
        index = report_list[:5].index(max(report_list[:5]))
        report_list[5] = ts_type[index]
    else:
        report_list[5] = ts_type[-1]

    # Add a subcategory: a special day seasonality that has a periodic impulse shape.
    if report_list[5] == 'hDay' or report_list[5] == 'Day':
        harmonic = set(range(12))
        all_peak = set()
        for counter, value in enumerate(indx):
            if amp[counter] > (mean + 3 * std):
                all_peak.add(int(round(cycle[value])))
        if len(harmonic.intersection(all_peak)) > 10:
            report_list[5] = ts_type[-2]

    report_list[-2] = y.mean()
    report_list[-1] = y.std()
    return report_list

In [13]:
# ts_0 = time_series_wiki_1[200].replace(to_replace=np.nan, value=0, inplace = False)
# ans1 = characterize_ts(ts_0,day)
# ans2 = characterize_ts(ts_0[-year:],day)
# print(ans1)
# print(ans2)
# discrete_ft(ts_0,day)
# discrete_ft(ts_0[-year:],day)

### Convert the report list to Pandas dataframe for easier speculation.

## Write to new JSON lines with cat

In [14]:
def series_to_obj(ts, cat=None):
    obj = {"start": str(ts.index[0]), "target": list(ts)}
    if cat is not None:
        obj["cat"] = cat
    return obj

def series_to_jsonline(ts, cat=None):
    return json.dumps(series_to_obj(ts, cat))

In [15]:
def cat_feature(score, boundary_12, bounadry_23):
    # Default category assignment.
    cat = 0 
    if score > 0:
        if score < boundary_12:
            cat = 1
        elif score < bounadry_23:
            cat = 2
        else:
            cat = 3
    return cat

In [16]:
def save_ts_cat(time_series, output_file):
    encoding = 'utf-8'
    with open(output_file, 'wb') as fp:
        for i, ts in enumerate(time_series):
            ts_0val = ts.replace(to_replace=np.nan, value=0, inplace = False)
            character_list = characterize_ts(ts_0val[-year:],day)
            cat = [0] * 3
            # Catagory for Trend characteristic
            cat[0] = cat_feature(character_list[0], 10, 20)
            # Category for HalfDay characteristic
            cat[1] = cat_feature(character_list[1], 10, 20)
            # Category for Day characteristic
            cat[2] = cat_feature(character_list[2], 20, 40)
            raw_jsons[i]['cat'] = cat
    with open(output_file, 'w') as outfile:
        outfile.write('\n'.join(json.dumps(i) for i in raw_jsons) + '\n') 

In [17]:
def save_ts_cat_1(time_series, output_file):
    encoding = 'utf-8'
    with open(output_file, 'wb') as fp:
        for i, ts in enumerate(time_series):
            ts_0val = ts.replace(to_replace=np.nan, value=0, inplace = False)
            character_list = characterize_ts(ts_0val[-year:],day)
            # Catagory for scale
            cat = [0]
            cat[0] = cat_feature(character_list[-2], 100, 1000) -1
            raw_jsons[i]['cat'] = cat
    with open(output_file, 'w') as outfile:
        outfile.write('\n'.join(json.dumps(i) for i in raw_jsons) + '\n') 

In [18]:
# save_ts_cat(time_series_wiki_1, output_file)
save_ts_cat_1(time_series_wiki_1, output_file)

In [19]:
#ts_wiki_2 = get_ts(output_file, freq)
test_df = pd.read_json(output_file, lines=True)
test_df.head(20)

,cat,start,target
0,[0],2016-01-01 00:00:00,"[5, 8, 9, 4, 13, 17, 18, 4, 7, 4, 23, 7, 5, 3,..."
1,[0],2016-01-01 00:00:00,"[1, None, 1, None, None, 1, 3, 2, 2, None, Non..."
2,[0],2016-01-01 00:00:00,"[None, 1, None, None, None, 1, 1, 1, None, 1, ..."
3,[1],2016-01-01 00:00:00,"[30, 39, 47, 26, 35, 37, 128, 91, 135, 114, 17..."
4,[0],2016-01-01 00:00:00,"[None, None, None, None, None, None, None, Non..."
5,[1],2016-01-01 00:00:00,"[36, 45, 81, 48, 31, 57, 251, 188, 204, 276, 4..."
6,[0],2016-01-01 00:00:00,"[None, None, None, None, None, None, None, Non..."
7,[2],2016-01-01 00:00:00,"[1356, 1130, 1069, 1013, 945, 1674, 1919, 1973..."
8,[0],2016-01-01 00:00:00,"[4, 4, 4, 7, 21, 3, 8, 9, 5, 5, 4, 4, 5, 6, 6,..."
9,[0],2016-01-01 00:00:00,"[14, 13, 13, 17, 6, 68, 27, 22, 27, 22, 45, 19..."
